In [30]:
import psycopg2
import pandas as pd
import json
from tqdm import tqdm_notebook

In [31]:
with open('config.json') as f:
    configDict = json.load(f)

In [32]:
with open('config_queries.json') as f2:
    configQueries = json.load(f2)

In [33]:
inserytQuery = configQueries['insertQuery']

In [34]:
user = configDict["user"]
password = configDict['password']
storage = configDict['storageip']
port = configDict['port']
database = configDict['database']

In [35]:
connection = psycopg2.connect(user = user,
                                  password = password,
                                  host = storage,
                                  port = port,
                                  database = database)

In [36]:
cur = connection.cursor()

cur.execute('select * from documents')
r = cur.fetchall()

In [37]:
r

[]

In [38]:
df = pd.read_pickle('../data/arxiv-last50years-data.pickle')

In [39]:
df.head()

,id,submitter,title,categories,abstract,update_date,authors_parsed,year,pdf_link,pages
0,0704.0001,Pavel Nadolsky,Calculation of prompt diphoton production cros...,High Energy Physics - Phenomenology,A fully differential calculation in perturba...,2008-11-26,"[[Balázs, C., ], [Berger, E. L., ], [Nadolsky,...",2007,https://arxiv.org/pdf/0704.0001,37 page
1,0704.0007,Alejandro Corichi,Polymer Quantum Mechanics and its Continuum Limit,General Relativity and Quantum Cosmology,A rather non-standard quantum representation...,2008-11-26,"[[Corichi, Alejandro, ], [Vukasinac, Tatjana, ...",2007,https://arxiv.org/pdf/0704.0007,16 page
2,0704.0009,Paul Harvey,"The Spitzer c2d Survey of Large, Nearby, Inste...",Astrophysics,We discuss the results from the combined IRA...,2010-03-18,"[[Harvey, Paul, ], [Merin, Bruno, ], [Huard, T...",2007,https://arxiv.org/pdf/0704.0009,N/A
3,0704.0015,Christian Stahn,Fermionic superstring loop amplitudes in the p...,High Energy Physics - Theory,The pure spinor formulation of the ten-dimen...,2009-11-13,"[[Stahn, Christian, ]]",2007,https://arxiv.org/pdf/0704.0015,22 page
4,0704.0016,Li Tong,Lifetime of doubly charmed baryons,High Energy Physics - Phenomenology,"In this work, we evaluate the lifetimes of t...",2008-12-18,"[[Chang, Chao-Hsi, ], [Li, Tong, ], [Li, Xue-Q...",2008,https://arxiv.org/pdf/0704.0016,17 page


In [40]:
df.shape

(134307, 10)

In [41]:
for i in tqdm_notebook(df.index):
    cur.execute(inserytQuery, [str(df.id[i]), str(df.submitter[i]),
                               str(df.title[i]), 
                               str(df.categories[i]),
                               str(df.abstract[i]), str(df.update_date[i]), str(df.authors_parsed[i]), int(df.year[i]),
                               str(df.pdf_link[i]), str(df.pages[i]), 'pipeline'
                              
                            ])
    
    if i%100==0:
        connection.commit()
    
connection.commit()

<ipython-input-41-dec957f2b9e9>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(df.index):


KeyboardInterrupt: 